The other two notebooks have been finein the sense that I learned a lot about the problem.

I got a sense for how candidate genertion feels, what trends might exist in the data, etc.

Let's not try generating a submission implementing a full (albeit small) data processing pipeline where at the end we will train an lgbm model and make a submission.

Let's get started.

In [1]:
import pandas as pd
import numpy as np
import arrow

In [3]:
%%time
# https://www.kaggle.com/paweljankiewicz/hm-create-dataset-samples

transactions = pd.read_csv('data/transactions_train.csv', dtype={"article_id": "str"})
customers = pd.read_csv('data/customers.csv')
articles = pd.read_csv('data/articles.csv', dtype={"article_id": "str"})

CPU times: user 20.7 s, sys: 1.83 s, total: 22.5 s
Wall time: 23.9 s


In [6]:
%%time

for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
    print(sample)
    customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
    customers_sample_ids = set(customers_sample["customer_id"])
    transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
    articles_sample_ids = set(transactions_sample["article_id"])
    articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
    customers_sample.to_csv(f"data/customers_sample_{sample_repr}.csv.gz", index=False)
    transactions_sample.to_csv(f"data/transactions_train_sample_{sample_repr}.csv.gz", index=False)
    articles_sample.to_csv(f"data/articles_train_sample_{sample_repr}.csv.gz", index=False)

0.001
0.01
0.05
CPU times: user 15.9 s, sys: 53.7 ms, total: 15.9 s
Wall time: 15.9 s


In [2]:
transactions = pd.read_csv('data/transactions_train_sample_01.csv.gz', dtype={"article_id": "str"})
customers = pd.read_csv('data/customers_sample_01.csv.gz')
articles = pd.read_csv('data/articles_train_sample_01.csv.gz', dtype={"article_id": "str"})

We can skil customers and articles for now, the most heart of the problem are the transactions.

We want to use features we generate for up to an including week $t$ to generate predictions for week $t_{t+1}$.

Of course, that is just one way to structure the problem. We could treat it as a time series problem where we look at the sequence of weeks $t_1...t_n$ and predict the purchases for week $t_{n+1}$. There are many ways to frame the problem.

But we want to do something that would be
* simple
* has a chance to lending itself well for using gradient boosted trees

In line with [the suggestion from Pawel Jankiewicz](https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/307288), we will predict baskets. And to make things even simpler, we will aggregate purchases by weeks.

In [61]:
transactions['week'] = transactions.t_dat \
    .apply(lambda t: arrow.get(t)) \
    .apply(lambda t: t.year * 100 + t.week) \
    .rank(method='dense').astype('int')

transactions.drop(columns='t_dat', inplace=True)

In [62]:
transactions['purchased'] = 1 # our positive examples

In [45]:
bestsellers_previous_week = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').head(20) \
    .groupby('week').rank(method='dense', ascending=False) \
    .to_frame('bestseller_previous_week_rank').reset_index()

bestsellers_previous_week.week += 1

In [47]:
transactions.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [59]:
negative_examples_bestsellers = pd.merge(transactions[['customer_id', 'week', 'price', 'sales_channel_id']], bestsellers_previous_week, how='outer', on='week')
negative_examples_bestsellers.head()

,customer_id,week,price,sales_channel_id,article_id,bestseller_rank
0,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,2,0.033881,2.0,0451229014,1.0
1,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,2,0.033881,2.0,0651335004,1.0
2,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,2,0.033881,2.0,0658848001,1.0
3,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,2,0.033881,2.0,0666143001,1.0
4,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,2,0.033881,2.0,0671515003,1.0


In [65]:
negative_transactions = transactions.copy()

In [70]:
negative_transactions.shape

(28534, 6)

In [69]:
negative_transactions[negative_transactions.week==2]

,customer_id,article_id,price,sales_channel_id,week,purchased
0,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0613497006,0.033881,2,2,1
1,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0474138001,0.033881,2,2,1
2,2e50a35485613ab376d1d1acd3a1efef779fcfe80bca46...,0686631001,0.042356,1,2,1
3,4670f2d4f48201418c5f79af70ae2dd5e53affacb21b4a...,0514134001,0.067780,1,2,1
4,4670f2d4f48201418c5f79af70ae2dd5e53affacb21b4a...,0639685006,0.042356,1,2,1
...,...,...,...,...,...,...
152,c4fc5bd374b7d2f21df0aac7e17e705c9b9d3897a7248c...,0446224002,0.013542,2,2,1
153,c4fc5bd374b7d2f21df0aac7e17e705c9b9d3897a7248c...,0609024002,0.028797,2,2,1
154,c4fc5bd374b7d2f21df0aac7e17e705c9b9d3897a7248c...,0639478002,0.013542,2,2,1
155,c4fc5bd374b7d2f21df0aac7e17e705c9b9d3897a7248c...,0393447010,0.040661,2,2,1


In [109]:
negative_transactions

,customer_id,article_id,price,sales_channel_id,week,purchased
0,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0613497006,0.033881,2,2,1
1,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0474138001,0.033881,2,2,1
2,2e50a35485613ab376d1d1acd3a1efef779fcfe80bca46...,0686631001,0.042356,1,2,1
3,4670f2d4f48201418c5f79af70ae2dd5e53affacb21b4a...,0514134001,0.067780,1,2,1
4,4670f2d4f48201418c5f79af70ae2dd5e53affacb21b4a...,0639685006,0.042356,1,2,1
...,...,...,...,...,...,...
32187,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0905811003,0.033881,2,107,1
32188,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0801673001,0.016932,2,107,1
32189,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0886566001,0.033881,2,107,1
32190,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0905811002,0.033881,2,107,1


In [185]:
negative_transactions.customer_id = negative_transactions.groupby('week')['customer_id'].transform(np.random.permutation)

In [188]:
negative_transactions.purchased = 0

In [193]:
transactions = pd.concat([transactions, negative_transactions]) \
    .drop_duplicates(['customer_id', 'article_id', 'week'], keep='first')

In [208]:
negative_examples_bestsellers = negative_examples_bestsellers[negative_examples_bestsellers.week != negative_examples_bestsellers.week.max()]

In [ ]:
transactions = pd.concat([transactions, negative_transactions]) \
    .drop_duplicates(['customer_id', 'article_id', 'week'], keep='first')

In [210]:
transactions = pd.concat([transactions, negative_examples_bestsellers])

In [213]:
transactions.purchased.fillna(0, inplace=True)

In [217]:
transactions = transactions[~transactions.bestseller_rank.isna()]

In [221]:
transactions.reset_index(drop=True, inplace=True)

In [222]:
transactions

,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank
0,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0451229014,0.033881,2.0,2,0.0,1.0
1,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0651335004,0.033881,2.0,2,0.0,1.0
2,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0658848001,0.033881,2.0,2,0.0,1.0
3,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0666143001,0.033881,2.0,2,0.0,1.0
4,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0671515003,0.033881,2.0,2,0.0,1.0
...,...,...,...,...,...,...,...
568689,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0508691005,0.033881,2.0,107,0.0,2.0
568690,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0509091058,0.033881,2.0,107,0.0,2.0
568691,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0537119003,0.033881,2.0,107,0.0,2.0
568692,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0547780001,0.033881,2.0,107,0.0,2.0


In [104]:
transactions

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
0,2018-09-20,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0613497006,0.033881,2,2.0,1
1,2018-09-20,05d0c63e8a3ff46f9519e38f1af70007d474650975ef40...,0474138001,0.033881,2,2.0,1
2,2018-09-20,2e50a35485613ab376d1d1acd3a1efef779fcfe80bca46...,0686631001,0.042356,1,2.0,1
3,2018-09-20,4670f2d4f48201418c5f79af70ae2dd5e53affacb21b4a...,0514134001,0.067780,1,2.0,1
4,2018-09-20,4670f2d4f48201418c5f79af70ae2dd5e53affacb21b4a...,0639685006,0.042356,1,2.0,1
...,...,...,...,...,...,...,...
32187,2020-09-22,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0905811003,0.033881,2,107.0,1
32188,2020-09-22,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0801673001,0.016932,2,107.0,1
32189,2020-09-22,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0886566001,0.033881,2,107.0,1
32190,2020-09-22,db1f0328dcdc4e12318b7091159ea412af2934047fc191...,0905811002,0.033881,2,107.0,1


In [35]:
cust_spend_per_week transactions.groupby(['customer_id', 'week'])['price'].sum()

customer_id                                                       week   
0012d60b384f6e693d4341629e5dc7211051800aaaf452dcdf9419ab5ab38a08  2020522    0.050814
002f0f63ede7b86c5226830252ffc07c6790d0b7ce6857965a6986de66475497  2020044    0.059305
00b6dea0bf44fc9e2807461b0201b9d8611c1f99477d340dfde98f7e792a45a2  2019148    0.051814
00f0f5d8cb94ecab029c46f63e89aac4fcf86dcec5f85ce50b026e0777ad8020  2019104    0.042339
                                                                  2019206    0.130458
                                                                               ...   
ff53bd3b2a37eeb34c252fdb0a7db240755a6d2581b9f1679a47c19495a55bc6  2020627    0.067780
                                                                  2020730    0.025390
ffddf1dc0462bedf4156c6bac6eb6a75d258a787aa48983f018836848ea9b8e0  2020417    0.098254
                                                                  2020835    0.166034
ffe28cc97d8e4e1acb3085c0a00bc988c4374266f9fe6681f6bb565373aa9bc2  

In [31]:
for grp in transactions.groupby(['customer_id', 'week']): break

In [33]:
grp[1]

,t_dat,customer_id,article_id,price,sales_channel_id,week
26691,2020-05-28,0012d60b384f6e693d4341629e5dc7211051800aaaf452...,0796210012,0.025407,2,2020522
26692,2020-05-28,0012d60b384f6e693d4341629e5dc7211051800aaaf452...,0751664001,0.025407,2,2020522


In [25]:
type(ts[0].year)

int